In [ ]:
# Устанавливаем библиотеки
# ! pip install pandas scikit-learn nltk spacy
# ! python -m spacy download ru_core_news_sm

In [35]:
# Подключаем необходимые для работы библиотеки
import numpy as np
import pandas as pd
import os
import gdown
import zipfile
import warnings
from google.colab import drive
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import nltk
import spacy
import joblib

In [3]:
# Отключаем предупреждения
warnings.filterwarnings('ignore')

# Подготовка данных

In [6]:
# Монтируем Google-диск для скачивания ZIP-файла, распаковки и работы с CSV-файлом
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Задаем имя ZIP-файла с записями о контрактах
name_zip = 'fz.zip'

In [8]:
# Скачиваем ZIP-файл с записями о контрактах с Google-диска
url = 'https://drive.google.com/uc?id=1sRHx27O3NgTivrrQHdBdTAqxdCYNmARW'
gdown.download(url, name_zip, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1sRHx27O3NgTivrrQHdBdTAqxdCYNmARW
From (redirected): https://drive.google.com/uc?id=1sRHx27O3NgTivrrQHdBdTAqxdCYNmARW&confirm=t&uuid=ce7c008e-9090-4b0d-a1cb-49306808d3c7
To: /content/fz.zip
100%|██████████| 2.47G/2.47G [00:44<00:00, 55.7MB/s]


'fz.zip'

In [9]:
# Распаковываем ZIP-файл с записями о контрактах
with zipfile.ZipFile(name_zip, 'r') as zip_ref:
  zip_ref.extractall()

# Предобработка данных

In [10]:
df_original = pd.DataFrame()
# Задаем имя CSV-файла с записями о контрактах
name_csv = 'fz.csv'
# Определяем типы необходимых записей для анализа
type_cols = {'Subject': 'string', 'OKPD2': 'string'}
# Читаем данные из супербольшого CSV-файла в датафрейм кусками, одновременно удаляя ненужные для анализа столбцы и строки с пустыми значениями
with pd.read_csv(name_csv, chunksize=100000, encoding='utf-8', on_bad_lines="skip", delimiter=',', header=None, usecols=[11, 26], names=['Subject', 'OKPD2'], dtype=type_cols) as reader:
  for chunk in reader:
    # Отфильтровываем записи о контрактах, оставляя записи групп ОКПД 41, 42, 43, 71.1
    chunk = chunk[(chunk['OKPD2'].str.len() <= 4) & (chunk['OKPD2'].str.contains('^41.') | chunk['OKPD2'].str.contains('^42.') | chunk['OKPD2'].str.contains('^43.') | chunk['OKPD2'].str.contains('^71.1'))]
    # Удаляем из датафрейма строки с неопределенным предметом закупки и строки с пустыми значениями
    chunk = chunk.loc[chunk['Subject'] != '<НЕ ОПРЕДЕЛЕНО>']
    chunk.dropna
    # Добавляем текущий chunk в итоговый большой датафрейм
    df_original = pd.concat([df_original, chunk], ignore_index=True)

In [11]:
# Создаем рабочий датафрейм
df_work = df_original

In [12]:
# Сбрасываем индексы в датафрейме
df_work = df_work.reset_index()
del df_work['index']

# Предварительный анализ полученного датафрейма

In [31]:
# 10 случайных записей
df_work.sample(10)

,Subject,OKPD2
787739,Выполнение работ по замеру сопротивления изоля...,43.2
1455676,Разработка проектной документации \Капитальный...,71.1
595718,Ремонт и техническое обслуживание сетей улично...,43.2
1737437,"Ремонт тротуара по адресу: ул. Совхозная, с. З...",42.1
1740295,Разработка рабочей документации и выполнение с...,41.2
807801,Ремонт дороги ул. Хариса Юсупова и ул. Соснова...,42.1
238388,Установка индивидуальный приборов учета потреб...,43.2
1141506,Выполнение работ по огнезащитной обработке дер...,43.2
1763066,Выполнение работ по благоустройству территории...,42.9
10588,№ 971 - А Выполнение работ по ремонту автомоби...,42.1


In [32]:
# Размерность
df_work.shape

(1764006, 2)

In [33]:
# Типы данных
df_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1764006 entries, 0 to 1764005
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Subject  string
 1   OKPD2    string
dtypes: string(2)
memory usage: 26.9 MB


In [ ]:
# Количество записей, сгруппированных по коду ОКПД2
df_work.groupby('OKPD2', as_index=False).agg({'Subject':'count'})

# Лемматизация предмета закупки

In [51]:
# Загружаем русскую модель spaCy
nlp_Subject = spacy.load('ru_core_news_sm')

In [52]:
# Определим функцию для лемматизации предмета закупки
def lemmatize_Subject(text_Subject):
  doc = nlp_Subject(text_Subject)
  return ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

In [ ]:
# Лемматизируем столбец с предметом закупки
df_work['lemmatized_Subject_of_Contract'] = df_work['Subject'].apply(lemmatize_Subject)

# Преобразование кодов ОКПД2 в текстовые метки

In [ ]:
# Определим функцию для преобразования кодов ОКПД2 в текстовые метки
def convert_OKPD2_to_text_labels(code_OKPD2):
  codes_OKPD2 = {
      '41': 'Здания и работы по возведению зданий',
      '41.1': 'Документация проектная для строительства',
      '41.2': 'Здания и работы по возведению зданий',
      '42': 'Сооружения и строительные работы в области гражданского строительства',
      '42.1': 'Дороги автомобильные и железные; строительные работы по строительству автомобильных дорог и железных дорог',
      '42.2': 'Сооружения и строительные работы по строительству инженерных коммуникаций',
      '42.9': 'Сооружения и строительные работы по строительству прочих гражданских сооружений',
      '43': 'Работы строительные специализированные',
      '43.1': 'Работы по сносу зданий и сооружений и по подготовке строительного участка',
      '43.2': 'Работы электромонтажные, работы по монтажу водопроводных и канализационных систем и прочие строительно-монтажные работы',
      '43.3': 'Работы завершающие и отделочные в зданиях и сооружениях',
      '43.9': 'Работы строительные специализированные прочие',
      '71.1': 'Услуги в области архитектуры, инженерно-технического проектирования и связанные технические консультативные услуги'
  }
  return [codes_OKPD2.get(c) for c in code_OKPD2.split(',') if c in codes_OKPD2]

In [ ]:
# Промаркируем датафрейм метками
df_work['text_Lables'] = df_work['OKPD2'].apply(convert_OKPD2_to_text_labels)

# Преобразование текстовых меток в бинарный формат

In [ ]:
# Для многоклассовой классификации используем бинарное представление текстовых меток
mlb_Labels = MultiLabelBinarizer()
y = mlb_Labels.fit_transform(df_work['text_Lables'])

# Разделение данных на обучающую и тестовую выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_work['lemmatized_Subject_of_Contract'], y, test_size=0.2, random_state=42)

# Создание модели обучения

In [ ]:
# Используем TfidfVectorizer для преобразования текста в числовые вектора
tfidf = TfidfVectorizer(max_features=10000)
# Используем метод логистической регрессии в качестве модели
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
# Построим пайплайн модели обучения
pipeline = Pipeline([
    ('tfidf', tfidf),
    ('clsf', model)
])

# Обучение модели на обучающей выборке

In [ ]:
pipeline.fit(X_train, y_train)

# Оценка модели на тестовой выборке

In [ ]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=mlb_Labels.classes_))

# Сохранение модели

In [ ]:
joblib.dump(pipeline, 'fz_class_model.pkl')

# Применение модели

In [ ]:
# Загружаем сохраненную модель
pipeline = joblib.load('fz_class_model.pkl')
# Определеим функцию для классификации нового контракта
def classify_new_contract(Subject_of_new_contract):
  lemmatized_Subject = lemmatize_Subject(Subject_of_new_contract)
  prediction = pipeline.predict([lemmatized_Subject])
  labels = mlb_Labels.inverse_transform(prediction)
  return labels[0]

# Пример использования

In [ ]:
new_contract = "Ремонт автомобильной дороги"
predicted_labels = classify_new_contract(new_contract)
print(predicted_labels)